In [ ]:
import numpy as np
from functools import lru_cache

# Initialize a 6x6 game board with zeros
board = np.zeros((6, 6), dtype=int)

# Constants for the game
EMPTY = 0
WHITE = 1
BLACK = 2
WINNING_LENGTH = 5

# Depth limit for the MiniMax algorithm
DEPTH_LIMIT = 1  # Set this value based on your needs

# Direction constants for rotation
CLOCKWISE = "R"
COUNTERCLOCKWISE = "L"

def print_board(board):
    for row in board:
        print(' '.join(str(cell) for cell in row))
    print()

def check_winner(board):
    for row in range(6):
        for col in range(6):
            if col + WINNING_LENGTH <= 6:
                if np.all(board[row, col:col+WINNING_LENGTH] == WHITE):
                    return WHITE
                if np.all(board[row, col:col+WINNING_LENGTH] == BLACK):
                    return BLACK
            if row + WINNING_LENGTH <= 6:
                if np.all(board[row:row+WINNING_LENGTH, col] == WHITE):
                    return WHITE
                if np.all(board[row:row+WINNING_LENGTH, col] == BLACK):
                    return BLACK
            if row + WINNING_LENGTH <= 6 and col + WINNING_LENGTH <= 6:
                if np.all([board[row+i, col+i] == WHITE for i in range(WINNING_LENGTH)]):
                    return WHITE
                if np.all([board[row+i, col+i] == BLACK for i in range(WINNING_LENGTH)]):
                    return BLACK
            if row - WINNING_LENGTH + 1 >= 0 and col + WINNING_LENGTH <= 6:
                if np.all([board[row-i, col+i] == WHITE for i in range(WINNING_LENGTH)]):
                    return WHITE
                if np.all([board[row-i, col+i] == BLACK for i in range(WINNING_LENGTH)]):
                    return BLACK
    return None

def rotate_subgrid(board, top_left, direction):
    subgrid = board[top_left[0]:top_left[0]+3, top_left[1]:top_left[1]+3]
    if direction == CLOCKWISE:
        subgrid = np.rot90(subgrid, -1)
    elif direction == COUNTERCLOCKWISE:
        subgrid = np.rot90(subgrid)
    board[top_left[0]:top_left[0]+3, top_left[1]:top_left[1]+3] = subgrid
    return board

@lru_cache(maxsize=None)
def minimax(board_tuple, depth, alpha, beta, maximizing_player):
    board = np.array(board_tuple).reshape(6, 6)
    winner = check_winner(board)
    if winner == WHITE:
        return 100 - depth
    elif winner == BLACK:
        return depth - 100
    elif np.all(board != EMPTY):
        return 0
    elif depth >= DEPTH_LIMIT:
        return evaluate_board(board)

    if maximizing_player:
        max_eval = -float('inf')
        for move in get_possible_moves(board, WHITE):
            _, _, move_board, _ = move
            eval = minimax(tuple(map(tuple, move_board)), depth + 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_possible_moves(board, BLACK):
            _, _, move_board, _ = move
            eval = minimax(tuple(map(tuple, move_board)), depth + 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def evaluate_board(board):
    white_score = 0
    black_score = 0

    for row in range(6):
        for col in range(6):
            if col + WINNING_LENGTH <= 6:
                white_score += evaluate_line(board[row, col:col+WINNING_LENGTH], WHITE)
                black_score += evaluate_line(board[row, col:col+WINNING_LENGTH], BLACK)
            if row + WINNING_LENGTH <= 6:
                white_score += evaluate_line(board[row:row+WINNING_LENGTH, col], WHITE)
                black_score += evaluate_line(board[row:row+WINNING_LENGTH, col], BLACK)
            if row + WINNING_LENGTH <= 6 and col + WINNING_LENGTH <= 6:
                white_score += evaluate_line([board[row+i, col+i] for i in range(WINNING_LENGTH)], WHITE)
                black_score += evaluate_line([board[row+i, col+i] for i in range(WINNING_LENGTH)], BLACK)
            if row - WINNING_LENGTH + 1 >= 0 and col + WINNING_LENGTH <= 6:
                white_score += evaluate_line([board[row-i, col+i] for i in range(WINNING_LENGTH)], WHITE)
                black_score += evaluate_line([board[row-i, col+i] for i in range(WINNING_LENGTH)], BLACK)

    return white_score - black_score

def evaluate_line(line, player):
    score = 0
    count = np.count_nonzero(line == player)
    if count == 2:
        score += 10
    elif count == 3:
        score += 50
    elif count == 4:
        score += 200
    elif count == 5:
        score += 1000
    return score

def get_possible_moves(board, player):
    moves = []
    for i in range(6):
        for j in range(6):
            if board[i, j] == EMPTY:
                for r in [0, 3]:
                    for c in [0, 3]:
                        for direction in [CLOCKWISE, COUNTERCLOCKWISE]:
                            new_board = board.copy()
                            new_board[i, j] = player
                            new_board = rotate_subgrid(new_board, (r, c), direction)
                            moves.append((i, j, new_board, (r, c, direction)))
    return moves

def read_board_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            for j, char in enumerate(line.strip()):
                if char == '1':
                    board[i, j] = WHITE
                elif char == '2':
                    board[i, j] = BLACK
                else:
                    board[i, j] = EMPTY

def write_board_to_file(file_path, board):
    with open(file_path, 'w') as file:
        for row in board:
            file.write(''.join(str(cell) for cell in row) + '\n')

def make_move():
    global board  # Ensure board is accessible within this function
    while True:
        try:
            x = int(input("Enter row number (1-6): ")) - 1
            y = int(input("Enter column number (1-6): ")) - 1
            plane = int(input("Enter subgrid number (1-4): ")) - 1
            rotation = input("Enter rotation direction (L/R): ").upper()
            
            if not (0 <= x < 6 and 0 <= y < 6 and 0 <= plane < 4 and rotation in ['L', 'R']):
                print("Invalid input! Please enter valid coordinates and rotation.")
                continue
            
            # Check if the board position is empty
            if board[x, y] == EMPTY:
                # Update the board with the player's move
                board[x, y] = BLACK
                
                # Rotate the specified 3x3 subgrid
                if plane == 0:
                    top_left = (0, 0)
                elif plane == 1:
                    top_left = (0, 3)
                elif plane == 2:
                    top_left = (3, 0)
                elif plane == 3:
                    top_left = (3, 3)
                
                if rotation == 'L':
                    board = rotate_subgrid(board, top_left, COUNTERCLOCKWISE)
                elif rotation == 'R':
                    board = rotate_subgrid(board, top_left, CLOCKWISE)
                
                return True
            else:
                print(f"Position ({x + 1}, {y + 1}) is not empty.")
        
        except ValueError:
            print("Invalid input! Please enter integers for coordinates.")
        except KeyboardInterrupt:
            print("\nGame interrupted.")
            exit()

    return False


def play_game():
    global board
    current_player = WHITE
    move_count = 0
    
    while True:
        print_board(board)
        
        if current_player == WHITE:
            # Computer's turn
            print("Computer's turn (White)")
            best_move = None
            best_value = -float('inf')
            
            for move in get_possible_moves(board, WHITE):
                _, _, move_board, rotation = move
                move_value = minimax(tuple(map(tuple, move_board)), 0, -float('inf'), float('inf'), False)
                
                if move_value > best_value:
                    best_value = move_value
                    best_move = move
            
            if best_move:
                x, y, new_board, rotation = best_move
                board[:] = new_board
                
                move_count += 1
                winner = check_winner(board)
                if winner == WHITE:
                    print_board(board)
                    print("The computer wins!")
                    break
                elif winner == BLACK:
                    print_board(board)
                    print("Congratulations! You win!")
                    break
                elif move_count == 36:  # Board filled up
                    print_board(board)
                    print("It's a draw!")
                    break
        else:
            # User's turn
            print("Your turn (Black)")
            success = make_move()
            if success:
                move_count += 1
                winner = check_winner(board)
                if winner == BLACK:
                    print_board(board)
                    print("Congratulations! You win!")
                    break
                elif winner == WHITE:
                    print_board(board)
                    print("The computer wins!")
                    break
                elif move_count == 36:  # Board filled up
                    print_board(board)
                    print("It's a draw!")
                    break
            else:
                print("Invalid move. Please try again.")
        
        current_player = WHITE if current_player == BLACK else BLACK


input_file = 'pentago.txt'

# Read the board
read_board_from_file(input_file)



play_game()



0 2 0 2 0 0
0 2 0 2 1 0
0 1 0 2 2 0
2 0 1 1 0 0
1 0 2 0 1 0
0 0 0 0 1 0

Computer's turn (White)
0 2 0 0 0 0
0 2 0 0 1 2
0 1 0 2 2 2
2 0 1 1 1 0
1 0 2 0 1 0
0 0 0 0 1 0

Your turn (Black)


Enter row number (1-6):  1
Enter column number (1-6):  1
